Scipy LP

In [1]:
import numpy as np
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

$\max \sum v_{i} x_{i}$ и $\sum w_{i} x_{i} \leq C$ привести к виду $\operatorname{min} c^{T} x$ и $A x \leq b$

$c=-v, \ A=w^{T}, \ b=(C)
$

In [2]:
c =  -np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [3]:
A.shape

(1, 6)

In [4]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -52.5
       ineqlin:  marginals: array([-3.5])
  residual: array([0.])
         lower:  marginals: array([13.5, 29.5, 27. ,  0. , 18. , 11.5])
  residual: array([0. , 0. , 0. , 7.5, 0. , 0. ])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 0
         slack: array([0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0., 0., 0.])
  residual: array([inf, inf, inf, inf, inf, inf])
             x: array([0. , 0. , 0. , 7.5, 0. , 0. ])

CVXPY (решает задачи ЦЛП)

In [5]:
import cvxpy 

In [6]:
x = cvxpy.Variable(shape=n, integer = True)
# A = A.ravel() # Преобразуем размерность массива
constraint =   A @ x <= C # cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = x @ c #cvxpy.sum(cvxpy.multiply(x, c))

In [7]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
print(x.value)
problem.solve()


None


-49.0

По одному товару минимум

In [8]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A.ravel(), x)) <= C # A @ x <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c)) # x @c

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
print(x.value)
problem.value

[1. 0. 0. 1. 0. 1.]


-17.0

Pulp*

In [2]:
from pulp import *
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Users\mblyz\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


***

In [10]:
from scipy.optimize import linprog
import numpy as np
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

In [11]:
c = cost.flatten()
print(c) 

[2 5 3 7 7 6]


In [12]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


In [36]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140 -110 -150 -140]


In [14]:
linprog(c=c, A_ub=A, b_ub=b)

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: 1900.0
       ineqlin:  marginals: array([-3., -0., -5., -7., -6.])
  residual: array([0., 0., 0., 0., 0.])
         lower:  marginals: array([0., 1., 0., 2., 0., 0.])
  residual: array([110.,   0.,  70.,   0., 150.,  70.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 5
         slack: array([0., 0., 0., 0., 0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0., 0., 0.])
  residual: array([inf, inf, inf, inf, inf, inf])
             x: array([110.,   0.,  70.,   0., 150.,  70.])

***

In [37]:
c = np.array([[1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2], 
    [10, 3, 1000, 6, 20], 
    [19, 7, 6, 1000, 4], 
    [8, 2, 20, 4, 1000]])

c_ = c.ravel()
c_
A_ub = np.eye(25)
b_ub = np.ones(25)

In [38]:
A_eq = []
b_eq = np.ones(10)
num_vars = 25
matrix_ord = 5
for i in range(5):
    A_eq.append([0]*5*i + [1]*5 + [0]*5*(matrix_ord-i-1))
for j in range(5):
    A_eq.append(([0]*j + [1] + [0]*(matrix_ord-j-1))*matrix_ord)
A_eq


[[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]]

In [40]:
linprog(c_, A_eq=A_eq, b_eq=b_eq)

           con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([ 8.,  1., -0.,  4., -2., 10.,  3.,  2.,  6., -0.])
  residual: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
           fun: 32.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: array([9.820e+02, 1.000e+00, 0.000e+00, 5.000e+00, 0.000e+00, 1.000e+00,
       9.960e+02, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       9.980e+02, 0.000e+00, 2.000e+01, 5.000e+00, 0.000e+00, 0.000e+00,
       9.900e+02, 0.000e+00, 0.000e+00, 1.000e+00, 2.000e+01, 0.000e+00,
       1.002e+03])
  residual: array([ 0.,  0.,  1.,  0., -0.,  0.,  0., -0.,  1.,  0., -0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0., -0.,  0.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 11
         slack: array([], dtype=float64)
        status: 0
       suc

***

In [3]:
import cvxpy as cvx

In [36]:
c = np.array(
    [
        [0, 12, 10, 19, 8],
        [12, 0, 3, 7, 2],
        [10, 3, 0, 6, 20],
        [19, 7, 6, 0, 4],
        [8, 2, 20, 4, 0],
    ]
)
x = cvx.Variable(shape=(5, 5), boolean=True)

In [35]:
constraints

[Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ()))]

In [57]:
constraints = [cvx.sum(x,axis=0)==1, cvx.sum(x,axis=1) == 1] + [x[i,i] == 0 for i in range(5)]
obj = cvx.sum(cvx.multiply(x,c))
problem = cvx.Problem(cvx.Minimize(obj), constraints=constraints)
problem.solve()
x.value

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1., -0.,  0.],
       [ 0., -0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0., -0.],
       [ 1.,  0.,  0., -0.,  0.]])

In [58]:
cvx.sum(cvx.multiply(x.value, c)).value

32.0